In [1]:
from IPython.core.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [20]:
import datetime
import eqcatalog

In [3]:
full_cat = eqcatalog.rob.query_local_eq_catalog(region=(0, 8, 49, 52), event_type='all')
## Add converted moment magnitude column
moment_mags = full_cat.get_magnitudes('MW', Mrelation={})
for i in range(len(full_cat)):
    full_cat[i].set_mag('MWc', moment_mags[i])
full_cat.default_Mrelations = {}

In [4]:
date_slider = widgets.IntRangeSlider(
    value=[1910, 2020],
    min=1350,
    max=2020,
    step=1,
    description='Year range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
#display(date_slider)

In [28]:
start_date_picker = widgets.DatePicker(
    value=datetime.date(1985, 1, 1),
    description='Start date:',
    disabled=False
)
#display(start_date_picker)

In [22]:
end_date_picker = widgets.DatePicker(
    value = datetime.date.today(),
    description='End date:',
    disabled=False
)
#display(end_date_picker)

In [5]:
mtype_selector = widgets.Dropdown(
    options=['ML', 'MS', 'MW', 'MWc'],
    value='ML',
    description='Mag. type:',
    disabled=False,
)
#display(mtype_selector)

In [35]:
lon_slider = widgets.FloatRangeSlider(
    value=[0., 8.],
    min=-1.,
    max=9.,
    step=0.1,
    description='Lon range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [36]:
lat_slider = widgets.FloatRangeSlider(
    value=[49., 52.],
    min=48.,
    max=53.,
    step=0.1,
    description='Lat range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [6]:
mag_slider = widgets.FloatRangeSlider(
    value=[2., 7.0],
    min=-1.,
    max=7.0,
    step=0.1,
    description='Magnitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
#display(mag_slider)

In [7]:
depth_slider = widgets.IntRangeSlider(
    value=[0, 50],
    min=0,
    max=50,
    step=1,
    description='Depth range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
#display(depth_slider)

In [8]:
etype_selector = widgets.SelectMultiple(
    options=[('Earthquake', 'ke'),
            ('Suspected earthquake', 'se'),
            ('Induced event', 'ki'),
            ('Suspected induced event', 'si'),
            ('Quarry blast', 'qb'),
            ('Suspected quarry blast', 'sqb'),
            ('Controlled explosion', 'cb'),
            ('Suspected controlled explosion', 'scb'),
            ('Mine explosion', 'km'),
            ('Suspected mine explosion', 'sm')],
    value=['ke'],
    #rows=10,
    description='Event type:',
    disabled=False
)
#display(etype_selector)

In [9]:
plot_type_selector = widgets.Dropdown(
    options=['basemap', 'folium'],
    value='folium',
    description='Plot type:',
    disabled=False,
)
#display(plot_type_selector)

In [39]:
def subselect_catalog(plot_type, min_date, max_date, lon_range, lat_range,
                      mag_range, Mtype, depth_range, event_types):
    #min_date, max_date = year_range
    #region = (0, 8, 49, 52)
    region = lon_range + lat_range
    min_mag, max_mag = mag_range
    min_depth, max_depth = depth_range
    subcat = full_cat.subselect(start_date=min_date, end_date=max_date,
                               region=region,
                               Mmin=min_mag, Mmax=max_mag, Mtype=Mtype,
                               min_depth=min_depth, max_depth=max_depth,
                               attr_val=('event_type', list(event_types)))
    subcat.name = 'ROB Catalog'
    if plot_type == 'basemap':
        ax = subcat.plot_map(region=region, Mtype=Mtype, Mrelation={})
    elif plot_type == 'folium':
        map = subcat.get_folium_map(Mtype=Mtype, Mrelation={}, region=region)
        display(map)

In [40]:
interactive_plot = interactive(subselect_catalog,
                               plot_type=plot_type_selector,
                               #year_range=date_slider,
                               min_date=start_date_picker, max_date=end_date_picker,
                               lon_range=lon_slider, lat_range=lat_slider,
                               mag_range=mag_slider, Mtype=mtype_selector,
                               depth_range=depth_slider,
                               event_types=etype_selector)
#output = interactive_plot.children[-1]
#output.layout.height = '350px'
interactive_plot

interactive(children=(Dropdown(description=u'Plot type:', options=('basemap', 'folium'), value='basemap'), DatePicker(value=datetime.date(2000, 1, 1), description=u'Start date:'), DatePicker(value=datetime.date(2020, 7, 27), description=u'End date:'), FloatRangeSlider(value=(0.0, 8.0), continuous_update=False, description=u'Lon range:', max=9.0, min=-1.0, readout_format=u'.1f'), FloatRangeSlider(value=(49.0, 52.0), continuous_update=False, description=u'Lat range:', max=53.0, min=48.0, readout_format=u'.1f'), FloatRangeSlider(value=(2.0, 7.0), continuous_update=False, description=u'Magnitude:', max=7.0, min=-1.0, readout_format=u'.1f'), Dropdown(description=u'Mag. type:', options=('ML', 'MS', 'MW', 'MWc'), value='ML'), IntRangeSlider(value=(0, 50), continuous_update=False, description=u'Depth range:', max=50), SelectMultiple(description=u'Event type:', index=(0,), options=(('Earthquake', 'ke'), ('Suspected earthquake', 'se'), ('Induced event', 'ki'), ('Suspected induced event', 'si'), ('Quarry blast', 'qb'), ('Suspected quarry blast', 'sqb'), ('Controlled explosion', 'cb'), ('Suspected controlled explosion', 'scb'), ('Mine explosion', 'km'), ('Suspected mine explosion', 'sm')), value=('ke',)), Output()), _dom_classes=('widget-interact',))